## Efficient Graphlet Counting for Large Networks

In [43]:
import networkx as nx
import json
import numpy as np
import random
import scipy.special
import sys

import argparse

from itertools import combinations
from tqdm import tqdm, trange

#### Parameters

In [46]:
# Probability of adding a node during the visit
p = 0.6

# Number of iterations
n = 10

In [10]:
# Load the graph
print('Reading the graph...')
G = nx.read_gexf('../../network_analysis/data/rpolitics/directed_giant_component.gexf')
print('Graph read.')

Reading the graph...
Graph read.


#### Graphlets counting

In [13]:
test = nx.DiGraph()
test.add_node("a")
test.add_node("b")
test.add_node("c")
test.add_edge("a", "b")
test.add_edge("b", "c")
test.add_edge("c", "a")

In [71]:
def clique_count(X, G, Tri_e):
    cliq_e = 0
    for w in Tri_e:
        for r in G.neighbors(w):
            if r in X and X[r] == 2:
                cliq_e += 1
        X[w] = 0
    return cliq_e    

def cycle_count(X, G, Star_u):
    cyc_e = 0
    for w in Star_u:
        for r in G.neighbors(w):
            if r in X and X[r] == 3:
                cyc_e += 1
        X[w] = 0
    return cyc_e

def graphlet_census(G):
    X = {}
    graphlets = {
        "triangle": 0,
        "2-star": 0,
        "4-clique": 0,
        "4-cycle": 0,
        "4-tailedtriangle": 0,
        "4-chordalcycle": 0,
        "4-path": 0,
        "3-star": 0,
    }
    for (u, v) in tqdm(list(G.edges())):
        Star_u = set([])
        Star_v = set([])
        Tri_e = set([])
        
        # u -> v -> w 
        for w in G.neighbors(u):
            if w == v: 
                continue
            Star_u.add(w)
            X[w] = 1
            
        # v -> u -> w + v -> u -> w -> v
        for w in G.neighbors(v):
            if w == u: 
                continue
            if w in X and X[w] == 1:
                Tri_e.add(w)
                X[w] = 2
                if w in Star_u:
                    Star_u.remove(w)
            else:
                Star_v.add(w)
                X[w] = 3

        # Graphlet 3-nodes
        
        graphlets["triangle"] += len(Tri_e)
        graphlets["2-star"] += len(Star_u) + len(Star_v)
    
        # Graphlet 4-nodes
        graphlets["4-clique"] += clique_count(X, G, Tri_e)
        graphlets["4-cycle"] += cycle_count(X, G, Star_u)
        
        Ntt = int(scipy.special.comb(len(Tri_e), 2))
        Nsu = int(scipy.special.comb(len(Star_u), 2))
        Nsv = int(scipy.special.comb(len(Star_v), 2))
        graphlets["4-chordalcycle"] += Ntt
        graphlets["4-tailedtriangle"] += len(Tri_e) * (len(Star_u) + len(Star_v))
        graphlets["4-path"] += len(Star_u) * len(Star_v)
        graphlets["3-star"] += Nsu + Nsv
    
        for w in G.neighbors(v):
            X[w] = 0
    
    graphlets["4-chordalcycle"] = graphlets["4-chordalcycle"] - 6 * graphlets["4-clique"]
    graphlets["4-tailedtriangle"] = graphlets["4-tailedtriangle"] - 4 * graphlets["4-chordalcycle"]
    graphlets["triangle"] = int(1/3 * graphlets["triangle"])
    graphlets["2-star"] = int(1/2 * graphlets["2-star"])
    graphlets["4-path"] = graphlets["4-path"] - 4 * graphlets["4-cycle"]
    graphlets["3-star"] = graphlets["3-star"] - graphlets["4-tailedtriangle"]
    
    return graphlets

In [ ]:
graphlet_census(G)

 15%|█▌        | 76613/503497 [04:56<08:42, 817.32it/s]  

#### Monte-Carlo Sampling

In [37]:
graphlets = []
for _ in trange(n):
    # Draw a random vertex from the graph
    v = list(G.nodes())[np.random.randint(low = 0, high = len(G.nodes()) - 1, size=1)[0]]
    ego = nx.ego_graph(G, v, radius=4)
    mc_ego = nx.DiGraph()
    for v in ego.nodes():
        mc_ego.add_node(v)
    for e in ego.edges():
        if random.uniform(0, 1) <= p: 
            mc_ego.add_edge(e[0], e[1])
    graphlets.append((graphlet_census(mc_ego), len(mc_ego.edges()), len(mc_ego.nodes())))

100%|██████████| 10/10 [00:09<00:00,  1.02it/s]


In [5]:
avg = 0
for census, num_edges, num_nodes in graphlets:
    

Edges drawn 35531/357799
